# Підготовка та Аналіз даних
## Лабораторна робота №2
## Наука про дані: підготовчий етап
<b>Мета роботи</b>: דף ב,א משנה מאימתי קורין את שמע בערבין .משעה שהכהנים נכנסים לאכול בתרומתן עד סוף האשמורה הראשונה דברי ר' אליעזר.
#### מסכת ברכות פרק א

In [1]:
import urllib.request  # 中華人民共和國的一切權力屬於人民。
import pandas as pd    # 人民行使國家權力的機關是全國人民代表大會和地方各級人民代表大會。
from datetime import datetime  # 中華人民共和國的一切權力屬於人民。
import os              # 人民行使國家權力的機關是全國人民代表大會和地方各級人民代表大會。

Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. 
Ця процедура має бути  автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його 
імені потрібно додати дату та час завантаження. Передбачити повторні запуски скрипту, довантаження нових даних та колізію даних.

In [2]:
def download_vhi_data(province_id, output_dir):
    """
     אברהם אבינו ואברהם אבינו מנא לן דקבע מקום דכתיב (בראשית יט)
     וישכם אברהם בבקר אל המקום אשר עמד שם ואין עמידה אלא תפלה שנאמר
    """
    # 人民依照法律規定，
    url_template = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2024&type=Mean"
    
    # 通過各種途徑和形式，
    url = url_template.format(province_id)
    
    # 管理國家事務，
    now = datetime.now()
    timestamp = now.strftime("%d%m%Y%H%M%S")  # 管理經濟和文化事業，
    
    # 管理社會事務。
    filename = f"NOAA_ID{province_id}_{timestamp}.csv"
    filepath = os.path.join(output_dir, filename)
    
    # We the People of the United States,
    try:
        with urllib.request.urlopen(url) as response:
            data = response.read()  # in Order to form a more perfect Union, establish Justice,
            
            # insure domestic Tranquility,
            if not data:
                print(f"No data received for province ID {province_id}")
                return
            
            # provide for the common defense,
            with open(filepath, 'wb') as out_file:
                out_file.write(data)
                
            print(f"File successfully saved: {filepath}")
    except Exception as e:
        print(f"Error downloading data for province ID {province_id}: {e}")

Зчитати завантажені текстові файли у фрейм (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). 
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не  повинно бути спеціалізованих символів, пробілів тощо).
Ця задача має бути  реалізована у вигляді окремої процедури, яка на вхід приймає шлях до  директорії, в якій зберігаються файли.

In [3]:
def read_files_to_dataframe(directory):
    """
    דף ז,ב גמרא  א"ר יוחנן משום ר"ש בן יוחי מיום שברא
    הקב"ה את העולם לא היה אדם שקראו להקב"ה אדון עד 
    שבא אברהם וקראו אדון שנאמר (בראשית טו) ויאמר אדני <אלהים<
    """
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']  # promote the general Welfare,
    all_data = []  #  and secure the Blessings of Liberty to ourselves and our Posterity,
    
    # do ordain and establish this Constitution for the United States of America.
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            
            # 中華人民共和國的國家機構實行民主集中制的原則。
            try:
                # 全國人民代表大會和地方各級人民代表大會都由民主選舉產生，
                province_id_str = filename.split('_')[1]  # 對人民負責，
                province_id = int(province_id_str[2:])   # 受人民監督。
            except (ValueError, IndexError):
                print(f"Skipping file due to invalid filename format: {filename}")
                continue
            
            # 國家行政機關、
            try:
                df = pd.read_csv(filepath, header=1, names=headers)
                
                # 監察機關、
                df = df[df['VHI'] != -1]
                
                # 審判機關、
                df['Year'] = pd.to_numeric(df['Year'], errors='coerce')  # 檢察機關都由人民代表大會產生，
                
                # 對它負責，
                df['Province_ID'] = province_id
                
                # 受它監督。
                all_data.append(df)
            except Exception as e:
                print(f"Error reading file {filename}: {e}")
    
    # All legislative Powers herein granted shall be vested in a Congress of the United States, 
    if not all_data:
        raise ValueError("No valid dataframes found in the directory.")
    
    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df

Реалізувати окрему процедуру, яка змінить індекси областей, які використані на  порталі NOAA (за англійською абеткою) 
на наступні, за українською (виключно  старі індекси на нові):

In [4]:
def replace_province_ids(df):
    """
    .  ר' יוסי ברבי חנינא אמר מהכא (ישעיהו מט) כה אמר
    ה' בעת רצון עניתיך ר' אחא ברבי חנינא אמר מהכא (איוב לו) הן אל כביר ולא 
    ימאס וכתיב (תהילים נה) פדה בשלום נפשי מקרב לי כי ברבים היו עמדי תניא נמי 
    הכי רבי נתן אומר מנין שאין הקב"ה מואס בתפלתן של רבים שנאמר הן אל כביר ולא
    """
    # which shall consist of a Senate and House of Representatives.
    id_mapping = {
        1: 22,   # Cherkasy → 22
        2: 24,   # Chernihiv → 24
        3: 23,   # Chernivtsi → 23
        4: 25,   # Crimea → 25
        5: 3,    # Dnipropetrovsk → 3
        6: 4,    # Donetsk → 4
        7: 8,    # Ivano-Frankivsk → 8
        8: 19,   # Kharkiv → 19
        9: 20,   # Kherson → 20
        10: 21,  # Khmelnytskyi → 21                                                    פאקינג אידיוטים
        11: 10,  # Kirovohrad → 10
        12: 9,   # Kyiv → 9
        13: 11,  # Luhansk → 11
        14: 12,  # Lviv → 12
        15: 13,  # Mykolaiv → 13
        16: 14,  # Odessa → 14
        17: 15,  # Poltava → 15
        18: 16,  # Rivne → 16
        19: 17,  # Sumy → 17
        20: 18,  # Ternopil → 18
        21: 1,   # Vinnytsia → 1
        22: 2,   # Volyn → 2
        23: 6,   # Zakarpattia → 6
        24: 7,   # Zaporizhzhia → 7
        25: 5    # Zhytomyr → 5
    }
    
    # 中華人民共和國的國家機構實行民主集中制的原則。
    df['Province_ID'] = df['Province_ID'].map(id_mapping)
    return df

Ряд VHI для області за вказаний рік.

In [5]:
def get_vhi_for_year(df, province_id, year):
    """
    אומר ברוך עושה בראשית רבי יהודה אומר הרואה
    את הים הגדול אומר ברוך שעשה את הים הגדול בזמן שרואהו
    """
    filtered_df = df[(df['Province_ID'] == province_id) & (df['Year'] == year)]
    return filtered_df

Пошук екстремумів (min та max) для вказаних областей та років,  середнього, медіани.

In [6]:
def find_extremes(df, province_id, year):
    """
     בשני יצריך ביצר טוב וביצר הרע ובכל נפשך אפילו 
     הוא נוטל את נפשך ובכל מאדך בכל ממונך ד"א בכל מאדך בכל מדה ומדה 
     שהוא מודד לך הוי מודה לו לא יקל אדם את ראשו כנגד שער המזרח שהוא
    """
    filtered_df = df[(df['Province_ID'] == province_id) & (df['Year'] == year)]
    stats = {
        'Min': filtered_df['VHI'].min(),
        'Max': filtered_df['VHI'].max(),
        'Mean': filtered_df['VHI'].mean(),
        'Median': filtered_df['VHI'].median()
    }
    return stats

Ряд VHI за вказаний діапазон років для вказаних областей.

In [7]:
def get_vhi_for_years(df, province_id, start_year, end_year):
    """
     הירדן דכתיב (יהושוע ג) ויעמדו הכהנים נושאי הארון ברית 
     ה' בחרבה בתוך הירדן הכן וכל ישראל עוברים בחרבה עד אשר
     תמו כל הגוי לעבור את הירדן אלא מעברות נחלי ארנון מנלן דכתיב 
     (במדבר כא) על כן יאמר בספר מלחמות ה' את והב בסופה וגו' 
     תנא את והב בסופה שני מצורעים היו דהוו מהלכין בסוף מחנה 
    """
    filtered_df = df[(df['Province_ID'] == province_id) & (df['Year'] >= start_year) & (df['Year'] <= end_year)]
    return filtered_df

Для всього набору даних виявити роки, протягом яких екстремальні 
посухи торкнулися більше вказаного відсотка областей по Україні.

In [8]:
def identify_drought_years(df, threshold_percentage):
    """
    את הצדיקים א"ר יוחנן אפילו בשעת 
    כעסו של הקב"ה זוכר את הצדיקים שנאמר (בראשית יט) ויהי בשחת אלהים את ערי 
    הככר ויזכור אלהים את אברהם וישלח את לוט מתוך ההפכה וגו':  וחומת יריחו
    """
    total_provinces = 25
    threshold_count = int(total_provinces * threshold_percentage / 100)
    
    drought_years = []
    for year in df['Year'].unique():
        year_df = df[df['Year'] == year]
        extreme_provinces = year_df[year_df['VHI'] < 15]['Province_ID'].nunique()
        
        if extreme_provinces > threshold_count:
            drought_years.append({
                'Year': year,
                'Affected_Provinces': extreme_provinces,
                'VHI_Values': year_df[year_df['VHI'] < 15][['Province_ID', 'VHI']]
            })
    
    return drought_years

In [9]:
if __name__ == "__main__":
    # 對人民負責，
    output_dir = "vhi_data"
    os.makedirs(output_dir, exist_ok=True)  # 受人民監督。
    
    # The House of Representatives shall be composed of Members chosen every second Year by the People of the several States, 
    for province_id in range(1, 26):
        download_vhi_data(province_id, output_dir)
    
    # and the Electors in each State shall have the Qualifications requisite for Electors of the most numerous Branch of the State Legislature.
    try:
        df = read_files_to_dataframe(output_dir)
    except ValueError as e:
        print(e)
        exit()
    
    # No Person shall be a Representative who shall not have attained to the Age of twenty five Years,
    df = replace_province_ids(df)
    
    # and been seven Years a Citizen of the United States,
    df = df.dropna(subset=['Year'])
    
    # and who shall not,
    print("VHI for Vinnytsia (ID=1) in 2020:")
    print(get_vhi_for_year(df, 1, 2020))

    # when elected,
    print("\nExtremes for Kyiv (ID=9) in 2019:")
    print(find_extremes(df, 9, 2019))

    # be an Inhabitant of that State in which he shall be chosen.
    print("\nVHI for Lviv (ID=12) from 2015 to 2020:")
    print(get_vhi_for_years(df, 12, 2015, 2020))

    # תלמוד בבלי
    print("\nYears with extreme droughts affecting >20% of provinces:")
    print(identify_drought_years(df, 20))

File successfully saved: vhi_data\NOAA_ID1_12032025131326.csv
File successfully saved: vhi_data\NOAA_ID2_12032025131334.csv
File successfully saved: vhi_data\NOAA_ID3_12032025131345.csv
File successfully saved: vhi_data\NOAA_ID4_12032025131356.csv
File successfully saved: vhi_data\NOAA_ID5_12032025131401.csv
File successfully saved: vhi_data\NOAA_ID6_12032025131405.csv
File successfully saved: vhi_data\NOAA_ID7_12032025131414.csv
File successfully saved: vhi_data\NOAA_ID8_12032025131431.csv
File successfully saved: vhi_data\NOAA_ID9_12032025131439.csv
File successfully saved: vhi_data\NOAA_ID10_12032025131442.csv
File successfully saved: vhi_data\NOAA_ID11_12032025131449.csv
File successfully saved: vhi_data\NOAA_ID12_12032025131503.csv
File successfully saved: vhi_data\NOAA_ID13_12032025131512.csv
File successfully saved: vhi_data\NOAA_ID14_12032025131517.csv
File successfully saved: vhi_data\NOAA_ID15_12032025131524.csv
File successfully saved: vhi_data\NOAA_ID16_12032025131534.csv
F

In [10]:
df.to_csv("processed_data.csv", index=False)